# 10: Creating Simple Plots

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/10_simple_plots.ipynb)

This tutorial suggests different strategies for creating simple visualizations from your xsnow dataset using existing libraries.

## What You'll Learn

- Simple timeseries plots showing scalar properties
- Simple vertical line profiles
- Simple plots to inspect distributions
- 2D maps of scalar properties
- Interactive profile visualization using Niviz

> **Note**: This tutorial focuses on simple, straightforward plotting strategies. For more advanced visualization techniques, see **03_visualization.ipynb**.


## Installation (For Colab Users)

If you're using Google Colab, run the cell below to install xsnow and dependencies. If you're running locally and have already installed xsnow, you can skip this cell.


In [ ]:
%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow
# Install niviz for interactive visualizations (optional)
%pip install -q niviz


In [ ]:
import xsnow
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load sample data
ds = xsnow.single_profile_timeseries()

%matplotlib inline


## Part 1: Simple Timeseries Plots Showing Scalar Properties

Timeseries plots are perfect for tracking how scalar (profile-level) properties change over time. These are properties that don't vary by layer, such as total snow height (HS), total snow water equivalent (SWE), or mean temperature.


In [ ]:
# Example 1: Snow height (HS) over time
# Select a single location and realization
hs_series = ds['HS'].isel(location=0, slope=0, realization=0)
times = ds.coords['time'].values

# Create a simple line plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(times, hs_series.values, 'b-', linewidth=2, marker='o', markersize=4)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Snow Height (m)', fontsize=12)
ax.set_title('Snow Height Time Series', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Strategy: Multiple Scalar Properties on One Plot

You can compare multiple scalar properties by plotting them on the same axes or using subplots:


In [ ]:
# Compare multiple scalar properties
times = ds.coords['time'].values
profile_data = ds.isel(location=0, slope=0, realization=0)

# Get multiple scalar variables
hs = profile_data['HS'].values
# If SWE is available, use it; otherwise compute mean density
if 'SWE' in profile_data.data_vars:
    swe = profile_data['SWE'].values
    y2_label = 'SWE (mm)'
    y2_data = swe
else:
    # Use mean density as alternative
    mean_density = profile_data['density'].mean(dim='layer').values
    y2_label = 'Mean Density (kg/m³)'
    y2_data = mean_density

# Create figure with two y-axes
fig, ax1 = plt.subplots(figsize=(10, 5))

# First y-axis for snow height
color1 = 'tab:blue'
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Snow Height (m)', fontsize=12, color=color1)
ax1.plot(times, hs, color=color1, linewidth=2, label='Snow Height')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.grid(True, alpha=0.3)

# Second y-axis for SWE or density
ax2 = ax1.twinx()
color2 = 'tab:red'
ax2.set_ylabel(y2_label, fontsize=12, color=color2)
ax2.plot(times, y2_data, color=color2, linewidth=2, linestyle='--', label=y2_label)
ax2.tick_params(axis='y', labelcolor=color2)

# Title
plt.title('Snow Height and ' + y2_label + ' Over Time', fontsize=14, fontweight='bold')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


**Now You Try**:
- Plot a different scalar property over time (check what profile-level variables are available with `ds.data_vars`)
- Create separate subplots for multiple properties instead of using twin axes
- Add markers only at specific intervals (e.g., every 5th point)


## Part 2: Simple Vertical Line Profiles

Vertical line profiles show how properties change with depth in the snowpack. This is the classic "snow pit" visualization where depth is on the y-axis and the property value is on the x-axis.


In [ ]:
# Example: Temperature profile
# Select a single profile
profile = ds.isel(location=0, time=0, slope=0, realization=0)

# Get depth (z coordinate) - convert to positive depth for plotting
depth = -profile.coords['z'].values  # Make positive (depth from surface)
temperature = profile['temperature'].values

# Create simple line plot
fig, ax = plt.subplots(figsize=(6, 8))
ax.plot(temperature, depth, 'r-', linewidth=2)
ax.axvline(x=0, color='k', linestyle='--', alpha=0.3, label='Freezing Point')
ax.set_xlabel('Temperature (°C)', fontsize=12)
ax.set_ylabel('Depth from surface (m)', fontsize=12)
ax.set_title('Temperature Profile', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()
ax.invert_yaxis()  # Surface at top
plt.tight_layout()
plt.show()


### Strategy: Multiple Properties on One Profile

You can plot multiple properties on the same profile plot to compare them:


In [ ]:
# Plot temperature and density on the same profile
profile = ds.isel(location=0, time=0, slope=0, realization=0)
depth = -profile.coords['z'].values
temp = profile['temperature'].values
density = profile['density'].values

fig, ax1 = plt.subplots(figsize=(8, 8))

# Temperature on left axis
color1 = 'tab:red'
ax1.set_xlabel('Temperature (°C)', fontsize=12, color=color1)
ax1.set_ylabel('Depth from surface (m)', fontsize=12)
ax1.plot(temp, depth, color=color1, linewidth=2, label='Temperature')
ax1.axvline(x=0, color=color1, linestyle='--', alpha=0.3)
ax1.tick_params(axis='x', labelcolor=color1)
ax1.grid(True, alpha=0.3)
ax1.invert_yaxis()

# Density on right axis
ax2 = ax1.twiny()
color2 = 'tab:blue'
ax2.set_xlabel('Density (kg/m³)', fontsize=12, color=color2)
ax2.plot(density, depth, color=color2, linewidth=2, linestyle='--', label='Density')
ax2.tick_params(axis='x', labelcolor=color2)

plt.title('Temperature and Density Profile', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


**Now You Try**:
- Create a profile plot for a different time step
- Plot a different variable (e.g., grain size, if available)
- Try using markers instead of lines: change `'r-'` to `'ro'` for circles


## Part 3: Simple Plots to Inspect Distributions

Distribution plots help you understand the statistical properties of your data. They're useful for identifying outliers, understanding data ranges, and checking data quality.


In [ ]:
# Strategy 1: Histogram
# Get all density values across all layers, times, and locations
all_densities = ds['density'].values.flatten()
# Remove NaN values
all_densities = all_densities[~np.isnan(all_densities)]

fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(all_densities, bins=30, edgecolor='black', alpha=0.7)
ax.set_xlabel('Density (kg/m³)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Distribution of Snow Density', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()


### Strategy: Using Seaborn for Enhanced Distribution Plots

Seaborn provides more sophisticated distribution plotting options:


In [ ]:
# Histogram with KDE (Kernel Density Estimate)
fig, ax = plt.subplots(figsize=(8, 5))
sns.histplot(all_densities, bins=30, kde=True, ax=ax)
ax.set_xlabel('Density (kg/m³)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Distribution of Snow Density (with KDE)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


In [ ]:
# Box plot - useful for comparing distributions across categories
# Get density values for different time periods
n_times = min(10, ds.sizes['time'])  # Use first 10 time steps
density_by_time = []

for i in range(n_times):
    temp_densities = ds['density'].isel(time=i).values.flatten()
    temp_densities = temp_densities[~np.isnan(temp_densities)]
    if len(temp_densities) > 0:
        density_by_time.append(temp_densities)

fig, ax = plt.subplots(figsize=(10, 6))
ax.boxplot(density_by_time, labels=[f'Time {i}' for i in range(n_times)])
ax.set_xlabel('Time Step', fontsize=12)
ax.set_ylabel('Density (kg/m³)', fontsize=12)
ax.set_title('Density Distribution Across Time Steps', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Strategy: Violin Plots for Detailed Distribution Shape

Violin plots combine box plots with KDE to show the full distribution shape:


In [ ]:
# Prepare data for violin plot
# Get surface layer (layer 0) temperatures across all times
surface_temps = []
times_list = []
for i in range(min(10, ds.sizes['time'])):
    temp_vals = ds['temperature'].isel(location=0, layer=0, time=i).values
    if not np.isnan(temp_vals):
        surface_temps.append(temp_vals)
        times_list.append(f'T{i}')

if len(surface_temps) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    parts = ax.violinplot(surface_temps, positions=range(len(surface_temps)), 
                          showmeans=True, showmedians=True)
    ax.set_xticks(range(len(surface_temps)))
    ax.set_xticklabels(times_list)
    ax.set_xlabel('Time Step', fontsize=12)
    ax.set_ylabel('Surface Temperature (°C)', fontsize=12)
    ax.set_title('Surface Temperature Distribution Across Time', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()


**Now You Try**:
- Create a histogram for a different variable (e.g., temperature)
- Compare distributions across different locations instead of time steps
- Try a different plot type: `sns.boxplot()` or `sns.violinplot()` with seaborn


## Part 4: 2D Maps of Scalar Properties

2D maps visualize spatial patterns in your data. If you have data from multiple locations, you can create maps showing how properties vary across space.


In [ ]:
# Strategy 1: Simple scatter plot map
# If you have location coordinates, use them; otherwise use location indices
n_locations = ds.sizes.get('location', 1)

if n_locations > 1:
    # Get snow height for all locations at a specific time
    time_idx = 0
    hs_values = []
    location_names = []
    
    for i in range(n_locations):
        hs_val = ds['HS'].isel(location=i, time=time_idx, slope=0, realization=0).values
        if not np.isnan(hs_val):
            hs_values.append(hs_val)
            # Try to get location name, otherwise use index
            try:
                loc_name = ds.coords['location'].values[i]
                location_names.append(str(loc_name))
            except:
                location_names.append(f'Loc {i}')
    
    # Create simple scatter plot (using location index as x-coordinate)
    fig, ax = plt.subplots(figsize=(10, 6))
    scatter = ax.scatter(range(len(hs_values)), hs_values, c=hs_values, 
                        cmap='viridis', s=100, edgecolors='black', linewidth=1)
    ax.set_xlabel('Location', fontsize=12)
    ax.set_ylabel('Snow Height (m)', fontsize=12)
    ax.set_title(f'Snow Height Across Locations (Time Step {time_idx})', 
                fontsize=14, fontweight='bold')
    ax.set_xticks(range(len(location_names)))
    ax.set_xticklabels(location_names, rotation=45, ha='right')
    plt.colorbar(scatter, label='Snow Height (m)')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("This dataset has only one location. For 2D maps, you need data from multiple locations.")


### Strategy: Heatmap for Time-Location Patterns

If you have multiple locations and time steps, a heatmap can show patterns across both dimensions:


In [ ]:
# Create a heatmap showing snow height across locations and time
n_locations = ds.sizes.get('location', 1)
n_times = min(20, ds.sizes['time'])  # Limit to first 20 time steps for clarity

if n_locations > 1:
    # Prepare data matrix
    hs_matrix = np.zeros((n_locations, n_times))
    
    for i in range(n_locations):
        for j in range(n_times):
            hs_val = ds['HS'].isel(location=i, time=j, slope=0, realization=0).values
            hs_matrix[i, j] = hs_val if not np.isnan(hs_val) else 0
    
    # Create heatmap
    fig, ax = plt.subplots(figsize=(12, 6))
    im = ax.imshow(hs_matrix, cmap='viridis', aspect='auto', interpolation='nearest')
    
    # Set labels
    ax.set_xlabel('Time Step', fontsize=12)
    ax.set_ylabel('Location', fontsize=12)
    ax.set_title('Snow Height Heatmap (Location vs Time)', fontsize=14, fontweight='bold')
    
    # Set ticks
    ax.set_xticks(range(0, n_times, max(1, n_times//10)))
    ax.set_yticks(range(n_locations))
    try:
        location_labels = [str(ds.coords['location'].values[i]) for i in range(n_locations)]
        ax.set_yticklabels(location_labels)
    except:
        ax.set_yticklabels([f'Loc {i}' for i in range(n_locations)])
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Snow Height (m)', fontsize=12)
    
    plt.tight_layout()
    plt.show()
else:
    print("This dataset has only one location. For location-time heatmaps, you need data from multiple locations.")


### Strategy: Using Seaborn for Heatmaps

Seaborn provides enhanced heatmap functionality with better styling:


In [ ]:
# Using seaborn heatmap
n_locations = ds.sizes.get('location', 1)
n_times = min(15, ds.sizes['time'])

if n_locations > 1:
    # Prepare data
    hs_matrix = np.zeros((n_locations, n_times))
    for i in range(n_locations):
        for j in range(n_times):
            hs_val = ds['HS'].isel(location=i, time=j, slope=0, realization=0).values
            hs_matrix[i, j] = hs_val if not np.isnan(hs_val) else 0
    
    # Create heatmap with seaborn
    fig, ax = plt.subplots(figsize=(12, 6))
    sns.heatmap(hs_matrix, cmap='viridis', annot=False, fmt='.2f', 
                cbar_kws={'label': 'Snow Height (m)'}, ax=ax)
    ax.set_xlabel('Time Step', fontsize=12)
    ax.set_ylabel('Location', fontsize=12)
    ax.set_title('Snow Height Heatmap (Seaborn)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("This dataset has only one location. For heatmaps, you need data from multiple locations.")


**Now You Try**:
- Create a heatmap for a different variable (e.g., mean temperature)
- If you have geographic coordinates, create a proper geographic map using cartopy
- Experiment with different colormaps: `'plasma'`, `'coolwarm'`, `'RdYlBu'`


## Part 5: Interactive Profile Visualization Using Niviz

Niviz is a specialized library for creating interactive snow profile visualizations. It provides a more sophisticated way to explore profiles with zooming, panning, and interactive features.

> **Note**: Niviz may require additional dependencies. If installation fails, you can skip this section or install manually.


In [ ]:
# Try importing niviz
try:
    import niviz
    NIVIZ_AVAILABLE = True
    print("Niviz is available!")
except ImportError:
    NIVIZ_AVAILABLE = False
    print("Niviz is not available. You may need to install it separately.")
    print("Try: pip install niviz")


### Strategy: Basic Niviz Profile

Niviz works with profile data. Let's prepare xsnow data for Niviz:


In [ ]:
if NIVIZ_AVAILABLE:
    # Select a single profile
    profile = ds.isel(location=0, time=0, slope=0, realization=0)
    
    # Get depth and properties
    depth = -profile.coords['z'].values  # Convert to positive depth
    temperature = profile['temperature'].values
    density = profile['density'].values
    
    # Remove NaN values
    valid_mask = ~np.isnan(depth) & ~np.isnan(temperature) & ~np.isnan(density)
    depth_clean = depth[valid_mask]
    temp_clean = temperature[valid_mask]
    density_clean = density[valid_mask]
    
    # Create a simple interactive plot using matplotlib (Niviz may have specific API)
    # Note: The exact Niviz API may vary. This is a general approach.
    # For actual Niviz usage, refer to: https://ninanor.github.io/NIviz/
    
    print("Profile data prepared:")
    print(f"  Depth range: {depth_clean.min():.2f} to {depth_clean.max():.2f} m")
    print(f"  Temperature range: {temp_clean.min():.2f} to {temp_clean.max():.2f} °C")
    print(f"  Density range: {density_clean.min():.0f} to {density_clean.max():.0f} kg/m³")
    print("\nFor interactive visualization with Niviz, you would typically:")
    print("  1. Convert your data to Niviz's expected format")
    print("  2. Use niviz.Profile() or similar function")
    print("  3. Display with .show() or .plot()")
    print("\nSee Niviz documentation for exact API: https://ninanor.github.io/NIviz/")
else:
    print("Niviz is not available. Using matplotlib for interactive-like visualization instead.")
    
    # Fallback: Create an interactive matplotlib plot
    from matplotlib.widgets import Slider
    
    profile = ds.isel(location=0, time=0, slope=0, realization=0)
    depth = -profile.coords['z'].values
    temp = profile['temperature'].values
    
    fig, ax = plt.subplots(figsize=(8, 10))
    plt.subplots_adjust(bottom=0.15)
    
    # Initial plot
    line, = ax.plot(temp, depth, 'r-', linewidth=2)
    ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)
    ax.set_xlabel('Temperature (°C)', fontsize=12)
    ax.set_ylabel('Depth from surface (m)', fontsize=12)
    ax.set_title('Temperature Profile (Interactive)', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.invert_yaxis()
    
    # Add slider for time selection (if multiple times available)
    if ds.sizes['time'] > 1:
        ax_time = plt.axes([0.2, 0.02, 0.6, 0.03])
        time_slider = Slider(ax_time, 'Time', 0, ds.sizes['time']-1, 
                            valinit=0, valstep=1)
        
        def update_profile(val):
            time_idx = int(time_slider.val)
            new_profile = ds.isel(location=0, time=time_idx, slope=0, realization=0)
            new_depth = -new_profile.coords['z'].values
            new_temp = new_profile['temperature'].values
            line.set_data(new_temp, new_depth)
            ax.set_ylim(new_depth.min(), new_depth.max())
            ax.set_xlim(new_temp.min(), new_temp.max())
            fig.canvas.draw_idle()
        
        time_slider.on_changed(update_profile)
    
    plt.tight_layout()
    plt.show()


### Strategy: Preparing Data for Niviz

Niviz typically expects data in a specific format. Here's how to prepare xsnow data:


In [ ]:
# Example: Prepare data structure that Niviz might expect
# (Actual API may vary - check Niviz documentation)

profile = ds.isel(location=0, time=0, slope=0, realization=0)
depth = -profile.coords['z'].values
temp = profile['temperature'].values
density = profile['density'].values

# Create a dictionary or DataFrame that Niviz can use
import pandas as pd

# Remove NaN values
valid_mask = ~(np.isnan(depth) | np.isnan(temp) | np.isnan(density))
profile_df = pd.DataFrame({
    'depth': depth[valid_mask],
    'temperature': temp[valid_mask],
    'density': density[valid_mask]
})

print("Profile data prepared as DataFrame:")
print(profile_df.head(10))
print(f"\nTotal layers: {len(profile_df)}")

# If Niviz is available, you could use it like:
# profile = niviz.Profile(profile_df['depth'], profile_df['temperature'])
# profile.show()


**Now You Try**:
- If you have Niviz installed, try creating an actual interactive profile
- Experiment with different properties in the profile
- Create profiles for different time steps and compare them
- Check the Niviz documentation for advanced features: https://ninanor.github.io/NIviz/


## Summary

✅ **What we learned:**

1. **Simple timeseries plots**: Track scalar properties over time using basic line plots
2. **Vertical line profiles**: Visualize properties as a function of depth
3. **Distribution plots**: Use histograms, box plots, and violin plots to understand data distributions
4. **2D maps**: Create spatial visualizations using scatter plots and heatmaps
5. **Interactive visualizations**: Introduction to Niviz for interactive profile exploration

## Key Strategies

- **Timeseries**: Use `plt.plot()` with time on x-axis for tracking changes
- **Profiles**: Always invert y-axis (`ax.invert_yaxis()`) to show surface at top
- **Distributions**: Use histograms for single variables, box plots for comparisons
- **Maps**: Use scatter plots for point data, heatmaps for grid data
- **Interactivity**: Niviz provides specialized tools for snow profiles

## Library Recommendations

- **matplotlib**: Core plotting library, good for all basic plots
- **seaborn**: Enhanced statistical visualizations, better default styles
- **niviz**: Specialized interactive snow profile visualization
- **pandas**: Useful for data preparation and simple plotting

## Next Steps

- **03_visualization.ipynb**: More advanced visualization techniques
- **04_advanced_analysis.ipynb**: Advanced analysis and calculations
- **08_advanced_xarray_techniques.ipynb**: Advanced xarray operations for data manipulation

## Exercises

1. Create a timeseries plot showing snow height for all available locations
2. Plot temperature profiles for three different time steps on the same figure
3. Create a histogram comparing density distributions across different locations
4. If you have multiple locations, create a heatmap showing mean temperature
5. Try installing and using Niviz to create an interactive profile visualization
